In [35]:
import numpy as np
from numpy import genfromtxt
import pandas as pd
import time
from scipy.sparse import coo_matrix
from scipy.sparse import save_npz

<span>1.</span> Import data.

In [2]:
t1 = time.time()

data = []
counter = 0

f = open('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\media\\media\\magyar media.txt')

for line in f:
    data.append(line)
f.close()

print(round((time.time() - t1) / 60, 1))

0.6


In [3]:
len(data)

58161071

<span>2.</span> Extract page_ids.

In [3]:
page_ids = []  # a list of page_ids

for i in range(len(data)):
    page_ids.append(data[i].split("\t")[0])   

In [18]:
page_ids = sorted(list(set(page_ids))) # to find the distinct page_ids

<span>3.</span> Build a dictionary of page likes: keys are page_ids, values are lists of user_ids liking the page.

In [20]:
page_likes = {}

In [21]:
for element in page_ids:
    page_likes[element] = []

In [22]:
t1 = time.time()

for i in range(len(data)):
    key = data[i].split("\t")[0]
    user = data[i].split("\t")[3]
    page_likes[key].append(user)

print(round((time.time() - t1) / 60, 1))

2.0


In [25]:
for key in page_likes.keys():
    page_likes[key] = set(page_likes[key])

<span>4.</span> Building the sparse matrix.

In [26]:
# Build the sparse matrix with the number of common likers. Users should have liked both pages at least twice. 

r = 0 # the index of the row of the matrix
c = 0 # the index of the columns of the matrix
row_indices = []
column_indices = []
matrix_elements = []

t1 = time.time()

for key_r in page_likes.keys(): # key_r is the key for the rows
    x = page_likes[key_r]
    for key_c in page_likes.keys(): # key_c is the key for the columns
        if c >= r:
            y = page_likes[key_c]
            common_set = set(x)&set(y)
            common_set_size = len(common_set)
            if common_set_size > 0:
                row_indices.append(r)
                column_indices.append(c)
                matrix_elements.append(common_set_size)
        c = c + 1
    r = r + 1
    c = 0

print(round((time.time() - t1) / 60, 1))

0.8


In [33]:
row_indices_2 = row_indices
column_indices_2 = column_indices
matrix_elements_2 = matrix_elements

for i in range(0, len(row_indices)):
    if row_indices[i] != column_indices[i]:
        column_indices_2.append(row_indices[i])
        row_indices_2.append(column_indices[i])
        matrix_elements_2.append(matrix_elements[i])

In [36]:
size = 86

row2  = np.array(row_indices_2)
col2  = np.array(column_indices_2)
val2 = np.array(matrix_elements_2)
mx = coo_matrix((val2, (row2, col2)), shape=(size, size))

In [38]:
save_npz('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\media\\media\\media_pages_sparse_matrix.npz', mx)

In [39]:
print("Matrix fill-up =", round(len(val2) / 86**2, 3), ".")

Matrix fill-up = 0.965 .
